<a href="https://colab.research.google.com/github/thaitruong018/e_commerce_analysis/blob/main/6_Predict_next_month_transaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-up

In [1]:
# gg drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Import packages
import os
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
# path to data
path = '/content/gdrive/MyDrive/brazilian_ecommerce'

In [4]:
# read data
df_orders = pd.read_csv(os.path.join(path,'df_order_items_products_customers.csv'))
df_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_id,quantity,seller_id,shipping_limit_date,price,freight_value,product_category_name,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,2e7a8482f6fb09756ca50c10d7bfc047,08c5351a6aca1c1589a38f244edeee9d,shipped,2016-09-04,2016-10-07,2016-10-18,2016-11-09,2016-10-20,c1488892604e4ba5cff5b4eb4d595400,1,1554a68530182680ad5c8b042c3ab563,2016-10-26 18:25:19,39.99,31.67,moveis_decoracao,b7d76e111c89f7ebf14761390f0f7d17,69309,boa vista,RR
1,2e7a8482f6fb09756ca50c10d7bfc047,08c5351a6aca1c1589a38f244edeee9d,shipped,2016-09-04,2016-10-07,2016-10-18,2016-11-09,2016-10-20,f293394c72c9b5fafd7023301fc21fc2,1,1554a68530182680ad5c8b042c3ab563,2016-10-26 18:25:19,32.90,31.67,moveis_decoracao,b7d76e111c89f7ebf14761390f0f7d17,69309,boa vista,RR
2,35d3a51724a47ef1d0b89911e39cc4ff,27ab53f26192510ff85872aeb3759dcc,delivered,2016-10-04,2016-10-05,2016-10-14,2016-10-26,2016-12-20,c1488892604e4ba5cff5b4eb4d595400,1,1554a68530182680ad5c8b042c3ab563,2016-10-14 02:44:30,39.99,22.34,moveis_decoracao,f922896769e9517ea3c630f3c8de86d0,96832,santa cruz do sul,RS
3,c4f710df20f7d1500da1aef81a993f65,4b671f05b6eb9dc1d2c1bae9c8c78536,delivered,2016-10-10,2016-10-10,2016-10-18,2016-10-26,2016-12-14,c1488892604e4ba5cff5b4eb4d595400,1,1554a68530182680ad5c8b042c3ab563,2016-10-26 16:35:46,39.99,11.80,moveis_decoracao,0ecf7f65b5ff3b9e61b637e59f495e0a,30411,belo horizonte,MG
4,81e5043198a44ddeb226002ff55d8ad4,ddd15ef77c83eea8c534d2896173a927,delivered,2017-01-09,2017-01-09,2017-01-09,2017-02-24,2017-02-24,c1488892604e4ba5cff5b4eb4d595400,2,1554a68530182680ad5c8b042c3ab563,2017-01-13 00:37:18,38.50,15.56,moveis_decoracao,853ba75a0b423722ccf270eea3b4cfe4,21843,rio de janeiro,RJ


## Data

In [5]:
# create a dataframe contains necessary columns
data = df_orders[['order_id','customer_unique_id','product_id','order_purchase_timestamp',
                    'quantity','price'
                    ]].copy()
data.rename(columns={'order_purchase_timestamp':'purchased_date'},inplace=True)
data.head()

,order_id,customer_unique_id,product_id,purchased_date,quantity,price
0,2e7a8482f6fb09756ca50c10d7bfc047,b7d76e111c89f7ebf14761390f0f7d17,c1488892604e4ba5cff5b4eb4d595400,2016-09-04,1,39.99
1,2e7a8482f6fb09756ca50c10d7bfc047,b7d76e111c89f7ebf14761390f0f7d17,f293394c72c9b5fafd7023301fc21fc2,2016-09-04,1,32.90
2,35d3a51724a47ef1d0b89911e39cc4ff,f922896769e9517ea3c630f3c8de86d0,c1488892604e4ba5cff5b4eb4d595400,2016-10-04,1,39.99
3,c4f710df20f7d1500da1aef81a993f65,0ecf7f65b5ff3b9e61b637e59f495e0a,c1488892604e4ba5cff5b4eb4d595400,2016-10-10,1,39.99
4,81e5043198a44ddeb226002ff55d8ad4,853ba75a0b423722ccf270eea3b4cfe4,c1488892604e4ba5cff5b4eb4d595400,2017-01-09,2,38.50


In [6]:
# overal info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102425 entries, 0 to 102424
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   order_id            102425 non-null  object 
 1   customer_unique_id  102425 non-null  object 
 2   product_id          102425 non-null  object 
 3   purchased_date      102425 non-null  object 
 4   quantity            102425 non-null  int64  
 5   price               102425 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 4.7+ MB


In [7]:
# change data type 
data['purchased_date'] = pd.to_datetime(data['purchased_date'])
# define purchase year month
data['PurchaseYearMonth'] = data['purchased_date'].dt.to_period('M')

In [8]:
# check timeframe
data['purchased_date'].min(), data['purchased_date'].max()

(Timestamp('2016-09-04 00:00:00'), Timestamp('2018-09-03 00:00:00'))

In [9]:
# first 5 rows of dataframe
data.head()

,order_id,customer_unique_id,product_id,purchased_date,quantity,price,PurchaseYearMonth
0,2e7a8482f6fb09756ca50c10d7bfc047,b7d76e111c89f7ebf14761390f0f7d17,c1488892604e4ba5cff5b4eb4d595400,2016-09-04,1,39.99,2016-09
1,2e7a8482f6fb09756ca50c10d7bfc047,b7d76e111c89f7ebf14761390f0f7d17,f293394c72c9b5fafd7023301fc21fc2,2016-09-04,1,32.90,2016-09
2,35d3a51724a47ef1d0b89911e39cc4ff,f922896769e9517ea3c630f3c8de86d0,c1488892604e4ba5cff5b4eb4d595400,2016-10-04,1,39.99,2016-10
3,c4f710df20f7d1500da1aef81a993f65,0ecf7f65b5ff3b9e61b637e59f495e0a,c1488892604e4ba5cff5b4eb4d595400,2016-10-10,1,39.99,2016-10
4,81e5043198a44ddeb226002ff55d8ad4,853ba75a0b423722ccf270eea3b4cfe4,c1488892604e4ba5cff5b4eb4d595400,2017-01-09,2,38.50,2017-01


## Calculation and prediction

In [10]:
# calculate total sales
data['TotalPurchase'] = data['quantity'] * data['price']

In [11]:
# Remove customer ids, which have only records for the last month
# Split into 2 dataframes to check customer ids
data['PurchaseYearMonth'] = data['PurchaseYearMonth'].astype(str)
df_0818 = data[data['PurchaseYearMonth'] == '2018-08']  
df_others = data[data['PurchaseYearMonth'] != '2018-08'] # since we want to predict trans on 08/2018

In [12]:
# get lists of customer ids
cust_ids_0818 = df_0818['customer_unique_id'].unique().tolist()
cust_ids_others = df_others['customer_unique_id'].unique().tolist()
# define customer ids, who made a purchase during the last month only
new_cust_ids = list(set(cust_ids_0818) - set(cust_ids_others))
# remove the customer ids without previous purchasing activity
data = data[~data['customer_unique_id'].isin(new_cust_ids)]
# remove all transaction on 08-2018
data_X = data[data['PurchaseYearMonth'] != '2018-08']

In [13]:
# assume we're are at the 01/08/2018 and want to predict trans in this month
now = dt.datetime(2018, 8, 1)
# calculate recency, frequency, monetary
data_features = data_X.groupby('customer_unique_id').agg({'purchased_date': lambda date: (now - date.max()).days,
                                              'order_id':pd.Series.nunique,
                                              'TotalPurchase': 'sum',
                                              'quantity': ['mean', 'sum']
                                              }).reset_index()
# rename columns
data_features.columns = ['CustomerID', 'Recency', 'Frequency', 
                       'Monetary', 'QuantityAvg', 'QuantityTotal'
                      ]    
data_features.head()                                               

,CustomerID,Recency,Frequency,Monetary,QuantityAvg,QuantityTotal
0,0000366f3b9a7992bf8c76cfdf3221e2,83,1,129.90,1.0,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,86,1,18.90,1.0,1
2,0000f46a3911fa3c0805444483337064,509,1,69.00,1.0,1
3,0000f6ccb0745a6a4b88665a16c9f078,293,1,25.99,1.0,1
4,0004aac84e0df4da2b147fca70cf8255,260,1,180.00,1.0,1


In [14]:
# get monthly transactions for customers
cust_month_trans = pd.pivot_table(data=data,
                                  index=['customer_unique_id'],
                                  values='order_id',
                                  columns=['PurchaseYearMonth'],
                                  aggfunc=pd.Series.nunique,
                                  fill_value=0).reset_index()
# rename index column
cust_month_trans = cust_month_trans.rename_axis('index',axis=1)                                 
cust_month_trans.head()                                 

index,customer_unique_id,2016-09,2016-10,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09
0,0000366f3b9a7992bf8c76cfdf3221e2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0000f46a3911fa3c0805444483337064,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0000f6ccb0745a6a4b88665a16c9f078,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0004aac84e0df4da2b147fca70cf8255,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [15]:
# define y and X for further model training and prediction
cols = [col for col in data_features.columns if col !='CustomerID']
X = data_features[cols]
y = cust_month_trans[['2018-08']]

In [16]:
X

,Recency,Frequency,Monetary,QuantityAvg,QuantityTotal
0,83,1,129.90,1.0,1
1,86,1,18.90,1.0,1
2,509,1,69.00,1.0,1
3,293,1,25.99,1.0,1
4,260,1,180.00,1.0,1
...,...,...,...,...,...
89178,419,1,1570.00,1.0,2
89179,234,1,64.89,1.0,1
89180,540,1,89.90,1.0,1
89181,91,1,115.00,1.0,1


In [17]:
y

index,2018-08
0,0
1,0
2,0
3,0
4,0
...,...
89178,0
89179,0
89180,0
89181,0


In [18]:
y.value_counts()

2018-08
0          89009
1            170
2              3
3              1
dtype: int64

In [19]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 42)

In [20]:
# Define the pipeline
pipeline = Pipeline([('scaler', StandardScaler()), 
                           ('model', LinearRegression())                           
                          ])    

In [21]:
# train model
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('model',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [22]:
# make predictions
train_pred_y = pipeline.predict(X_train)
test_pred_y = pipeline.predict(X_test)

In [23]:
# calculate MAE using scikit-learn
print("MAE:",mean_absolute_error(y_train,train_pred_y))
print("MSE",mean_squared_error(y_train, train_pred_y))
# compute the RMSE of our predictions
print("RMSE:",np.sqrt(mean_squared_error(y_train, train_pred_y)))

MAE: 0.0037875899265526988
MSE 0.0019180325327291562
RMSE: 0.0437953483001238


In [24]:
# calculate MAE using scikit-learn
print("MAE:",mean_absolute_error(y_test,test_pred_y))
print("MSE",mean_squared_error(y_test, test_pred_y))
# compute the RMSE of our predictions
print("RMSE:",np.sqrt(mean_squared_error(y_test, test_pred_y)))

MAE: 0.004548733253973248
MSE 0.002958016857975107
RMSE: 0.05438765354356728


In [27]:
df_pred = pd.DataFrame(test_pred_y, columns=['prediction'])
df_pred.head()

,prediction
0,0.002028
1,0.002428
2,0.000888
3,0.001886
4,0.001192


In [28]:
df_pred.value_counts()

prediction
 0.001421     15
 0.001340      8
 0.001426      7
 0.001582      6
 0.002129      5
              ..
 0.001838      1
 0.001838      1
 0.001838      1
 0.001838      1
-0.010047      1
Length: 16539, dtype: int64